# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [82]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import string
string.punctuation

from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\q514907\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\q514907\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\q514907\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
# load data from database
engine = create_engine('sqlite:///UdacityDisasterResponse.db')
df = pd.read_sql_table("Messages_Categories", con=engine)

df = df[:1000]

# df = pd.read_sql_table("Messages_Categories", con="sqlite:///DisasterResponse.db")
X = df["message"]
Y = df.iloc[:,4:].astype("int")

df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [51]:
def tokenize(text):
    messages_tokenized = []
    
    for index, message in enumerate(text):
        message = message.lower()
        
        # Remove punctuation
        # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
        message = message.translate(str.maketrans('', '', string.punctuation))
        
        # Tokenize into sentences and then words
        sentence_list = sent_tokenize(message)
        
        try:
            words_tokenized = [word_tokenize(sentence) for sentence in sentence_list][0]
        except:
            words_tokenized = message
#             print(Exception)
            print(f"Message {message} at index {index} caused an error. Message not tokenzied.")
            

        # Remove stopwords
        words_tokenized = [w for w in words_tokenized if w not in stopwords.words("english")]
        
        messages_tokenized.append(words_tokenized)
        
    return messages_tokenized


In [52]:
def tokenize_single(text):
    
    message = text.lower()
    
    # Remove punctuation
    # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    message = message.translate(str.maketrans('', '', string.punctuation))
        
    # Tokenize into sentences and then words
    sentence_list = sent_tokenize(message)
        
    try:
        words_tokenized = [word_tokenize(sentence) for sentence in sentence_list][0]
    except:
        words_tokenized = [""]
        print(Exception)
        print(f"Text '{text}' replaced with empty string.")

    # Remove stopwords
    words_tokenized = [w for w in words_tokenized if w not in stopwords.words("english")]
    
    
    return words_tokenized


In [53]:
test_text = ["Upper", "Upper2"][0]
# test_text = [][0]

test_text.lower()



'upper'

In [54]:
df["message"][0].translate(str.maketrans('', '', string.punctuation))

df["message"][0].lower()

'weather update - a cold front from cuba that could pass over haiti'

In [55]:
df["message"][:10]
df["message"][0]

tokenize_single(df["message"][0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [56]:
# tokenized_list = tokenize(df["message"][:10])
# tokenized_list = tokenize(df["message"])


# tokenized_list
# print("Done.")

In [57]:
# tokenized_list[7591]
# df["message"][7590:7592]

# df.iloc[7591]

In [58]:
def lemmatize(text):
    """
    This function takes a series of text pieces, tokenizes by words via the tokenize() function and then lemmatizes each token.
    The output is a list of lemmatized tokens per message.
    """
    # tokenize text
    messages_tokenized = tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    messages_lemmatized = []
    
    for message in messages_tokenized:
        try:
            words_lemmatized = [lemmatizer.lemmatize(token) for token in message]
            messages_lemmatized.append(words_lemmatized)
            max_index = messages_tokenized.index(message)
        except:
            print(f"Message at index {max_index + 1} caused an error. Message not lemmatized.")
            

    return messages_lemmatized

In [59]:
def lemmatize_single(text):
    """
    This function takes a series of text pieces, tokenizes by words via the tokenize() function and then lemmatizes each token.
    The output is a list of lemmatized tokens per message.
    """
    # tokenize text
    message_tokenized = tokenize_single(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    words_lemmatized = [lemmatizer.lemmatize(token) for token in message_tokenized]
    
    # Apply porter stemmer
    ps = PorterStemmer()
    words_stemmed = [ps.stem(word) for word in words_lemmatized]
    
    return words_stemmed

In [60]:
tokenize_single(df["message"][10])

lemmatize_single(df["message"][10])

['there', 'noth', 'eat', 'water', 'starv', 'thirsti']

In [61]:
testing = lemmatize_single(df["message"][10])

ps = PorterStemmer()

[ps.stem(word) for word in testing]

['there', 'noth', 'eat', 'water', 'starv', 'thirsti']

In [62]:
lemmatize_single(df["message"][10])

['there', 'noth', 'eat', 'water', 'starv', 'thirsti']

In [63]:
test_lem = lemmatize(df["message"][:10])

print("Done.")
test_lem

Done.


[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'],
 ['hurricane'],
 ['looking', 'someone', 'name'],
 ['un',
  'report',
  'leogane',
  '8090',
  'destroyed',
  'hospital',
  'st',
  'croix',
  'functioning',
  'need',
  'supply',
  'desperately'],
 ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'],
 ['information', 'national', 'palace'],
 ['storm', 'sacred', 'heart', 'jesus'],
 ['please', 'need', 'tent', 'water', 'silo', 'thank'],
 ['would', 'like', 'receive', 'message', 'thank'],
 ['croixdesbouquets',
  'health',
  'issue',
  'worker',
  'santo',
  '15',
  'area',
  'croixdesbouquets']]

In [64]:
# all_messages_lemmatized = lemmatize(df["message"])
all_messages_lemmatized = lemmatize(X[:10])

print("Done.")
all_messages_lemmatized

Done.


[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'],
 ['hurricane'],
 ['looking', 'someone', 'name'],
 ['un',
  'report',
  'leogane',
  '8090',
  'destroyed',
  'hospital',
  'st',
  'croix',
  'functioning',
  'need',
  'supply',
  'desperately'],
 ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'],
 ['information', 'national', 'palace'],
 ['storm', 'sacred', 'heart', 'jesus'],
 ['please', 'need', 'tent', 'water', 'silo', 'thank'],
 ['would', 'like', 'receive', 'message', 'thank'],
 ['croixdesbouquets',
  'health',
  'issue',
  'worker',
  'santo',
  '15',
  'area',
  'croixdesbouquets']]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [74]:
from sklearn.pipeline import Pipeline
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import hamming_loss


In [66]:
# print(len(all_messages_lemmatized))
# print(X.shape[0])

In [67]:
Y.sum(axis=1)
categories_colnames = list(Y.columns)
categories_colnames

X_subset = X[:2]
Y_subset = Y.head(2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# X_train, X_test, Y_train, Y_test = train_test_split(X_subset, Y_subset)

# Y_train["request"]

print("Done.")

Done.


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor


# # categories_colnames = ["related"] #, "request"]

# # https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

# pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize_single)),
# #         ('tfidf', TfidfVectorizer(lowercase=False)),
#         ('tfidf', TfidfTransformer()),

#         ('clf', OneVsRestClassifier(MultinomialNB(
#             fit_prior=True, class_prior=None))),
#     ])

# model_dict = {}

# for category in categories_colnames:
#     print(f'... Processing {category}')
#     # train the model using X_dtm & y
#     pipeline.fit(X_train, Y_train[category])
#     # compute the testing accuracy
#     prediction = pipeline.predict(X_test)
#     print(f'Test accuracy is {accuracy_score(Y_test[category], prediction)}')
#     model_dict[category] = pipeline

In [69]:
# print(classification_report(Y_test["related"], prediction))
# print(classification_report(Y_test["related"], model_dict["related"].predict(X_test)))



In [70]:
stop_code

NameError: name 'stop_code' is not defined

# Test with MultiOutputClassifier()

In [ ]:
Y["related"].unique()
Y.loc[Y["related"]==2]

In [ ]:
for col in Y:
    print(Y[col].unique())

In [ ]:

pipeline_OvR = Pipeline([
        ('vect', CountVectorizer(tokenizer=lemmatize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', OneVsRestClassifier(RandomForestClassifier()
            )),
    ])

# train the model using X_dtm & y
pipeline_OvR.fit(X_train, Y_train)
# compute the testing accuracy
prediction_OvR = pipeline_OvR.predict(X_test)

In [ ]:
# pipeline.score(X_test, Y_test)

print(classification_report(Y_test, prediction_OvR))

# Random Forest

In [ ]:
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor

pipeline_RF = Pipeline([
        ('vect', CountVectorizer(tokenizer=lemmatize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(RandomForestClassifier(
            ))),
    ])

# train the model using X_dtm & y
pipeline_RF.fit(X_train, Y_train)
# compute the testing accuracy
prediction_RF = pipeline_RF.predict(X_test)

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


In [ ]:
from sklearn.metrics import classification_report
# pipeline.score(X_test, Y_test)

# print(classification_report(Y_test, prediction_RF))
print(classification_report(Y_test, prediction_RF, target_names=categories_colnames))

                        precision    recall  f1-score   support

               related       0.85      0.98      0.91       206
               request       0.86      0.86      0.86       147
                 offer       0.00      0.00      0.00         1
           aid_related       0.80      0.82      0.81       154
          medical_help       1.00      0.06      0.12        31
      medical_products       1.00      0.06      0.11        18
     search_and_rescue       0.00      0.00      0.00        10
              security       0.00      0.00      0.00         5
              military       0.00      0.00      0.00         3
                 water       0.90      0.86      0.88        44
                  food       0.89      0.86      0.88        65
               shelter       0.92      0.29      0.44        41
              clothing       0.00      0.00      0.00         7
                 money       0.00      0.00      0.00         9
        missing_people       0.00      

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [ ]:

hamming_loss(Y_test, prediction_RF)

0.058057142857142856

In [ ]:
stop_code

In [71]:

pipeline_NB = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(MultinomialNB(
            fit_prior=True, class_prior=None))
            ),
    ])

# train the model using X_dtm & y
pipeline_NB.fit(X_train, Y_train)
# compute the testing accuracy
prediction_NB = pipeline_NB.predict(X_test)

print(classification_report(Y_test, prediction_NB, target_names=categories_colnames))

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.
                        precision    recall  f1-score   support

               related       0.78      1.00      0.87      4946
               request       0.90      0.22      0.36      1124
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.63      0.69      2732
          medical_help       0.00      0.00      0.00       536
      medical_products       1.00      0.00      0.01       369
     search_and_rescue       0.00      0.00      0.00       187
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       212
                 water       0.00      0.00      0.00       426
                  food       0.63      0.02      0.03       758
               shelter       0.00      0.00      0.00       606
              clothing       0.00      0.00      0.00       110
                 money       0.00      0.00

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [75]:
hamming_loss(Y_test, prediction_NB)

0.06570067399942918

In [ ]:
df_refugees = df.loc[df["death"] == 1]

df_refugees["message"].iloc[13]

'We are a few kilometers in the road to jacmel, Please come help us'

# SVM

In [76]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

pipeline_SVM = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(SGDClassifier(
                                        ))
            ),
    ])

# train the model using X_dtm & y
pipeline_SVM.fit(X_train, Y_train)
# compute the testing accuracy
prediction_SVM = pipeline_SVM.predict(X_test)

print(classification_report(Y_test, prediction_SVM))
hamming_loss(Y_test, prediction_SVM)

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4946
           1       0.81      0.56      0.67      1124
           2       0.00      0.00      0.00        30
           3       0.78      0.67      0.72      2732
           4       0.65      0.16      0.26       536
           5       0.77      0.16      0.27       369
           6       0.72      0.07      0.13       187
           7       0.00      0.00      0.00       108
           8       0.68      0.09      0.16       212
           9       0.73      0.61      0.67       426
          10       0.79      0.71      0.75       758
          11       0.84      0.49      0.62       606
          12       0.75      0.42      0.54       110
          13       0.67      0.03      0.05       149
          14       1.00      0.01      0.03        72
          15       0.83      0.08      0.15       225
          16       0.

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

0.05172012557904674

# Test Multilayer Perceptron MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MLPClassifier(random_state=1, max_iter=300)),
    ])

# train the model using X_dtm & y
pipeline.fit(X_train, Y_train)
# compute the testing accuracy
prediction = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
# pipeline.score(X_test, Y_test)

print(classification_report(Y_test, prediction))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
print("Test")

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 1

cv = 1

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [80]:
import pickle

pickle.dump()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.